# Project 3: **Ask‑the‑Web Agent**

Welcome to Project 3! In this project, you will learn how to use tool‑calling LLMs, extend them with custom tools, and build a simplified *Perplexity‑style* agent that answers questions by searching the web.

## Learning Objectives  
* Understand why tool calling is useful and how LLMs can invoke external tools.
* Implement a minimal loop that parses the LLM's output and executes a Python function.
* See how *function schemas* (docstrings and type hints) let us scale to many tools.
* Use **LangChain** to get function‑calling capability
* Combine LLM with a web‑search tool to build a simple ask‑the‑web agent.
* Connect to external tools using **MCP (Model Context Protocol)**, a universal standard for LLM‑tool integration.
* Optionally build a UI using Chainlit to test your agent.

## Roadmap
0. Environment setup
1. Write simple tools and connect them to an LLM
2. Standardize tool calling with JSON schemas
3. Use LangGraph for tool calling
4. Build a Perplexity-style web-search agent
5. (Optional) MCP: connect to external tool servers
6. (Optional) A minimal UI

# 0- Environment setup

### Step 1: Create your environment and install dependencies 
Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook, and use Conda or uv to install the project dependencies.

#### Option 1: Conda


```bash
# Create and activate the conda environment
conda env create -f environment.yml && conda activate web_agent

```

#### Option 2: UV (faster)

If you prefer [uv](https://docs.astral.sh/uv/) over Conda:

```bash
# Install uv (skip if already installed)
curl -LsSf https://astral.sh/uv/install.sh | sh

# Create venv and install dependencies
uv venv .venv-web-agent-uv && source .venv-web-agent-uv/bin/activate
uv pip install -r requirements.txt
```

### Step 2: Register this environment as a Jupyter kernel
```bash
python -m ipykernel install --user --name=web_agent --display-name "web_agent"
```
Now open your notebook and switch to the `web_agent` kernel (Kernel → Change Kernel).

### Step 3: Set up Ollama

In this project, we use **Ollama** to load and use open-weight LLMs. We start with smaller models like `gemma3:1b` and then switch to larger models like `llama3.2:3b`.

Start the **Ollama** server in a terminal. This launches a local API endpoint that listens for LLM requests.

```bash
ollama serve
```

Downloads the model so you can run them locally without API calls. 
```bash
ollama pull gemma3:1b
ollama pull llama3.2:3b
```

You can explore other available models [here](https://ollama.com/library) and pull them to experiment with.

In [4]:
# Quick check: is Ollama running?
# If this fails, open a terminal and run: ollama serve

import httpx

# response = httpx.get("http://localhost:11434/api/tags", timeout=5)
# models = [m["name"] for m in response.json().get("models", [])]
# print(f"Ollama is running. Installed models: {models}")

try:
    # vLLM default port is 8000
    response = httpx.get("http://localhost:11434/api/tags", timeout=5)
    models = [m["name"] for m in response.json().get("models", [])]
    print(f"Ollama is running. Installed models: {models}")
except Exception as e:
    print(f"Ollama is not running or unreachable at http://localhost:11434/api/tags")
    print(f"Error: {e}")
    print("\nTo run the ollama server, use the command below in your terminal.")

# Example using gemma-3-4b-it
# python -m vllm.entrypoints.openai.api_server --model google/gemma-3-4b-it

Ollama is running. Installed models: ['gemma3:4b', 'llama3.2:3b', 'gemma3:1b']


## 1- Tool Calling

LLMs are strong at answering questions, but they cannot directly access external data such as live web results, APIs, or computations. In real applications, agents rarely rely only on their internal knowledge. They need to query APIs, retrieve data, or perform calculations to stay accurate and useful. Tool calling bridges this gap by allowing the LLM to request actions from the outside world.

<img src="assets/tools.png" width="700">

As show below, We first implement a tool, then describe the tool as part of the model's prompt. When the model decides that a tool is needed, it emits a structured output. A parser will detect this output, execute the corresponding function, and feed the result back to the LLM so the conversation continues.

<img src="assets/tool_flow.png" width="700">

In this section, you will implement a simple `get_current_weather` function and teach the `gemma3:1b` model to use it when required.

In [5]:
# ---------------------------------------------------------
# Step 1: Implement the tool
# ---------------------------------------------------------
# You can either:
#   (a) Call a real weather API (for example, OpenWeatherMap), or
#   (b) Create a dummy function that returns a fixed response (e.g., "It is 23°C and sunny in San Francisco.")
#
# Output:
#   • Return a short, human-readable sentence describing the weather.
#
# Example expected behavior:
#   get_current_weather("San Francisco") → "It is 23°C and sunny in San Francisco."
#


# def get_current_weather(city):
#     return f"It is 23°C and sunny in {city}."

import os
from dotenv import load_dotenv

load_dotenv()

def get_weather_by_coords(lat: float, lon: float, units: str = "metric"):
    """
    Fetches current weather from OpenWeatherMap using latitude and longitude.
    """
    # Get your API key from an environment variable for security
    api_key = os.getenv("OPENWEATHER_API_KEY")
    
    if not api_key:
        return "Error: OPENWEATHER_API_KEY not found in environment variables."

    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units={units}"
    
    unitsSymbol  = ""
    if units == "metric":
        unitsSymbol = "C"
    else:
        unitsSymbol = "F"

    try:
        response = httpx.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        temp = data["main"]["temp"]
        desc = data["weather"][0]["description"]
        city = data.get("name", "at these coordinates")
        return f"It is {temp}°{unitsSymbol} and {desc} in {city}."
    except Exception as e:
        return f"Failed to fetch weather: {e}"


def get_coords_by_city(city_name):
    """
    Converts a city name into latitude and longitude using OpenWeatherMap's Geocoding API.
    """
    api_key = os.getenv("OPENWEATHER_API_KEY")
    # Geocoding endpoint
    geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name}&limit=1&appid={api_key}"
    
    try:
        response = httpx.get(geo_url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if not data:
            return None, None
        
        # Extract lat and lon from the first result
        return data[0]['lat'], data[0]['lon']
    except Exception as e:
        print(f"Geocoding error: {e}")
        return None, None

def get_current_weather(city):
    """
    The main tool function: 1. Gets Coords -> 2. Gets Weather
    """
    # 1. Convert City Name to Lat/Lon
    lat, lon = get_coords_by_city(city)
    
    if lat is None:
        return f"Sorry, I couldn't find the location: {city}"
    
    # 2. Get Weather using those coords (using your previous function)
    return get_weather_by_coords(lat, lon)

# Example Usage: # San Diego
print(get_weather_by_coords(47.6062, -122.3321, "imperial")) 

print(get_current_weather("seattle")) 




It is 57.72°F and broken clouds in Seattle.
It is 14°C and broken clouds in Seattle.


In [29]:
# ----------------------------------------------------------------------
# Step 2: Create a prompt to teach the LLM when and how to use your tool
# ----------------------------------------------------------------------
# What to include:
#   • A SYSTEM_PROMPT that tells the model about the tool use and describes the tool
#   • A USER_QUESTION with a user query that should trigger the tool.
#       Example: "What is the weather in San Diego today?"

# SYSTEM_PROMPT = """
#     You are a helpful assistant that can use a web search tool to answer user questions.
#     when the user asks a question, you should always check if the question can be answered using the web search tool.
#     if the question can be answered using the get_current_weather tool, use that tool to answer the question.
#     if the question cannot be answered using the web search tool, and get weather tool then answer the question using your knowledge,
#     and tell the user that you cannot answer the question using the web search tool.    
#     """
SYSTEM_PROMPT = """
You are a helpful assistant that can use tools.
When you need to use a tool, you MUST output a JSON object in exactly this format:
TOOL_CALL: {"name": "tool_name", "args": {"arg_name": "value"}}
Available Tools:
- get_current_weather(city): Get the current weather for a city.
Example:
User: What is the weather in Seattle?
Assistant: TOOL_CALL: {"name": "get_current_weather", "args": {"city": "Seattle"}}
"""
USER_QUESTION = "What is the weather in seattle today?"



Now that you have defined a tool and shown the model how to use it, the next step is to call the LLM using your prompt.

In [31]:
# ---------------------------------------------------------
# Step 3: Call the LLM with your prompt
# ---------------------------------------------------------
# Task:
#   Send SYSTEM_PROMPT + USER_QUESTION to the model.
#
# Steps:
#   1. Create an Ollama client
#   2. Use chat.completions.create to send your prompt to gemma3:1b
#   3. Print the response.
#
# Expected:
#   The model should return something like:
#   TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}
# ---------------------------------------------------------
import os
from openai import OpenAI

# Ollama runs on port 11434
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama", 
)
response = client.chat.completions.create(
    model="gemma3:4b",
    messages=[ {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_QUESTION}
    ],
    temperature = 0.7,
    stream=True,
)
output_text = ""
for chunk in response:
    if chunk.choices[0].delta.content:
        content = chunk.choices[0].delta.content
        output_text += content
        print(content, end="", flush=True)



TOOL_CALL: {"name": "get_current_weather", "args": {"city": "Seattle"}}


In [33]:
# ---------------------------------------------------------
# Step 4: Manually parse the LLM output and call the tool
# ---------------------------------------------------------
# Task:
#   Detect when the model requests a tool, extract its name and arguments,
#   and execute the corresponding function.
#
# Steps:
#   1. Search for the text pattern "TOOL_CALL:{...}" in the model output.
#   2. Parse the JSON inside it to get the tool name and args.
#   3. Call the matching function (e.g., get_current_weather).
#
# Expected:
#   You should see a line like:
#       Calling tool `get_current_weather` with args {'city': 'San Diego'}
#       Result: It is 23°C and sunny in San Diego.
# ---------------------------------------------------------

import re, json

# Task: 
# tool_call_regex = re.search(r"TOOL_CALL:\s*(\{.*?\})", output_text)
# (Removing the ? makes it "greedy" so it grabs everything until the last bracket).
# re.DOTALL flag, which allows . to match newlines, and re.IGNORECASE to handle slight variations.
tool_call_regex = re.search(r"TOOL_CALL:\s*(\{.*\})", output_text, re.DOTALL | re.IGNORECASE)   

if tool_call_regex:
    try:
        #Parse the json inside
        tool_data = json.loads(tool_call_regex.group(1))
        tool_name = tool_data["name"]
        tool_args = tool_data["args"]

        if tool_name == "get_current_weather":
            print(f"Calling tool `{tool_name}` with args {tool_args}")
            result = get_current_weather(**tool_args)
            print(f"Result: {result}")
        else:
            print(f"Found unknown tool call: {tool_name}")    

    except Exception as e:
        print(f"\nParsing Error: {e}")
        print(f"Raw string attempted: {tool_call_regex.group(1)}")
else:
    print("No tool call detected in the model output.")

# Debugging: Print the raw output to see why the regex failed
print("--- Debugging Tool Call Detection ---")
print(f"Raw model output:\n{repr(output_text)}")


Calling tool `get_current_weather` with args {'city': 'Seattle'}
Result: It is 14.9°C and clear sky in Seattle.
--- Debugging Tool Call Detection ---
Raw model output:
'TOOL_CALL: {"name": "get_current_weather", "args": {"city": "Seattle"}}\n'


# 2- Standardize tool calling

So far, we handled tool calling manually by writing a function, manually teaching the LLM about it, and write a regex to parse the output. This approach does not scale if we want to add more tools. Adding more tools would mean more `if/else` blocks and manual edits to the prompt.

To make the system flexible, we can standardize tool definitions by automatically reading each function's signature, converting it to a JSON schema, and passing that schema to the LLM. This way, the LLM can dynamically understand which tools exist and how to call them without requiring manual updates to prompts or conditional logic.

Next, you will implement a small helper that extracts metadata from functions and builds a schema for each tool.

In [44]:
# ---------------------------------------------------------
# Generate a JSON schema for a tool automatically
# ---------------------------------------------------------
#
# Steps:
#   1. Rewrite the get_current_weather function with docstring and arg types
#   2. Use `inspect.signature` to automatically get function parameters and docstring
#   2. For each argument, record its name, type, and description.
#   3. Build a schema containing: name, description, and parameters.
#   4. Test your helper on `get_current_weather` and print the result.
#
# Expected:
#   A dictionary describing the tool (its name, args, and types).
# ---------------------------------------------------------

from pprint import pprint
import inspect

def get_current_weather(city: str, unit: str = "metric") -> str:
    """Fetches the current weather for a city. You can specify 'celsius' or 'fahrenheit'.""" 
    lat, lon = get_coords_by_city(city)
    if lat is None:
        return f"Could not find coordinates for {city}"

    weather_info = get_weather_by_coords(lat, lon)
    return f"{weather_info} (Units requested: {unit})"

def to_schema(fn):
    tool_name = fn.__name__
    tool_description = inspect.getdoc(fn) or "No description provided"

    sig = inspect.signature(fn)

    parameters = {
        "type": "object",
        "properties": {},
        "required": [],
    }

    for param_name, param in sig.parameters.items():
        param_type = "string"
        if param.annotation == int:
            param_type = "ineteger"
        elif param.annotation == float:
            param_type = "number"

        parameters["properties"][param_name] = {
            "type": param_type,
            "desctiption": param_name
        }
    
    if param.default == inspect.Parameter.empty:
        parameters["required"].append(param_name)
    
    return {
        "name": tool_name,
        "description": tool_description,
        "paramaters": parameters
    }

tool_schema = to_schema(get_current_weather)
pprint(tool_schema)

{'description': 'Fetches the current weather for a city. You can specify '
                "'celsius' or 'fahrenheit'.",
 'name': 'get_current_weather',
 'paramaters': {'properties': {'city': {'desctiption': 'city',
                                        'type': 'string'},
                               'unit': {'desctiption': 'unit',
                                        'type': 'string'}},
                'required': [],
                'type': 'object'}}


In [52]:
# ---------------------------------------------------------
# Provide the tool schema to the model instead of prompt surgery
# ---------------------------------------------------------
# Goal:
#   Give the model a "menu" of available tools so it can choose
#   which one to call based on the user’s question.
#
# Steps:
#   1. Add an extra system message (e.g., name="tool_spec")
#      containing the JSON schema(s) of your tools.
#   2. Include SYSTEM_PROMPT and the user question as before.
#   3. Send the messages to the model (e.g., gemma3:1b).
#   4. Print the model output to see if it picks the right tool.
#
# Expected:
#   The model should produce a structured TOOL_CALL indicating
#   which tool to use and with what arguments.
# ---------------------------------------------------------

import json

# 1. Define tool schema and add system message
tools = [tool_schema]

# 2. Include SYSTEM_PROMPT and the user question
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "system", "name": "tool_spec", "content": json.dumps(tools)},
    {"role": "user", "content": USER_QUESTION}
]

# 3. Send the messages to the model
response = client.chat.completions.create(
    model="gemma3:4b", # Make sure to use the model you actually have (e.g. gemma3:4b or llama3.2:3b)
    messages=messages,
    temperature=0.7, # Lower temperature is often better for tool calling
    stream=True
)

# 4. Print the model output
output_text = ""
for chunk in response:
    if chunk.choices[0].delta.content:
        content = chunk.choices[0].delta.content
        output_text += content
        # print(content, end="", flush=True)
print(output_text)


TOOL_CALL: {"name": "get_current_weather", "args": {"city": "Seattle"}}


## 3- LangChain for Tool Calling

So far, you built a simple tool-calling pipeline. While this helps you understand the logic, it does not scale well when working with multiple tools, complex parsing, or multi-step reasoning. We have to write manual parsers, function calling logic, and adding responses back to the prompt.

LangChain simplifies this process. You only need to declare your tools, and its *Agent* abstraction handles when to call a tool, how to use it, and how to continue reasoning afterward. In this section, you will create a **ReAct** Agent (Reasoning + Acting). As shown below, the model alternates between reasoning steps and tool use wihtout any manual work.

<img src="assets/react.png" width="500">

The following links might be helpful for completing this section:
- [Create Agents](https://docs.langchain.com/oss/python/langchain/agents)
- [LangChain Tools](https://docs.langchain.com/oss/python/langchain/tools)
- [Ollama](https://docs.langchain.com/oss/python/integrations/chat/ollama)

In [54]:
# ---------------------------------------------------------
# Step 1: Define tools for LangChain
# ---------------------------------------------------------
# Steps:
#   1. Keep your existing `get_current_weather` function as before.
#   2. Create a new function (e.g., get_weather) that calls it.
#   3. Add the `@tool` decorator so LangChain can register it automatically.
#
# Notes:
#   • The decorator converts your Python function into a standardized tool object.

from langchain_core.tools import tool

def get_current_weather(location: str, unit: str = "fahrenheit") -> str:
    """Mock function to get the current weather."""
    return f"The current weather in {location} is {unit} degrees."

@tool
def get_weather(location: str = "San Francisco", unit: str = "fahrenheit") -> str:
    """Get the current weather in a given location."""
    return get_current_weather(location, unit)


In [64]:
# ---------------------------------------------------------
# Step 2: Create the Agent
# ---------------------------------------------------------
# Steps:
#   1. Create a gemma3:1b LLM instance 
#   2. Create the agent using create_agent
#   3. Test the agent with a natural question using agent.invoke

from langchain_ollama import ChatOllama
from langchain.agents import create_agent

llm = ChatOllama(
    model = "gemma3:1b",
    temperature = 0
)

agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

print(agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
))


ResponseError: registry.ollama.ai/library/gemma3:1b does not support tools (status code: 400)

### What just happened?
Your run failed because `gemma3:1b` does not support native tool calling (function calling). LangChain expects the model to return a structured tool-call object, but `gemma3:1b` can only return plain text, so the tool invocation step breaks.

### Why previosuly, our manual approach worked with any model?

In previous sections, we used **text-based tool calling**. We described the tool format in the system prompt. We asked the model to output `TOOL_CALL: {"name": ..., "args": ...}`. We then parsed this text with regex.

This works with **any model** (even small ones like `gemma3:1b`) because we're just asking the model to follow a certain structured output format.

### Why LangChain requires specific models?

LangChain relies on **native tool calling** and it expects a consistent structured output format irrespective of the model. Hence, it enfornces model outputs structured tool calls in a specific format. This requires models trained specifically for function calling

**Rule of thumb**: Models under 3B parameters typically lack native tool-calling capability.

| Model | Size | Native Tool Support | Notes |
|-------|------|---------------------|-------|
| `gemma3:1b` | 1B | No | Works for manual approach only |
| `llama3.2:1b` | 1B | No | Works for manual approach only |
| `llama3.2:3b` | 3B | Yes | Good balance of speed and capability |
| `gemma3` | 4B | Yes | Supports native tools |
| `mistral` | 7B | Yes | Strong tool support |

Let's fix the issue we observed in the previous cell.



In [63]:
# ---------------------------------------------------------
# Step 2 (retry): Re-create the Agent with a native tool-calling LLM
# ---------------------------------------------------------
# Steps:
#   1. Create a llama3.2:3b LLM instance 
#   2. Create a system prompt to teach react-style reasoning to the LLM
#   3. Create the agent using create_agent
#   4. Test the agent with a natural question using agent.invoke

from langchain_ollama import ChatOllama
from langchain.agents import create_agent

llm = ChatOllama(
    model = "llama3.2:3b",
    temperature = 0
)

agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

print(agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
))

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='048edb91-f871-45f2-9adb-deaf02be6e44'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2026-02-07T07:01:42.33624Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3128804500, 'load_duration': 1851772250, 'prompt_eval_count': 160, 'prompt_eval_duration': 843737250, 'eval_count': 22, 'eval_duration': 421490708, 'logprobs': None, 'model_name': 'llama3.2:3b', 'model_provider': 'ollama'}, id='lc_run--019c36e7-fd03-7f33-883d-d552be851a61-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'sf', 'unit': ''}, 'id': 'b62f6956-430d-412b-9e44-04f6d4046382', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 160, 'output_tokens': 22, 'total_tokens': 182}), ToolMessage(content='The current weather in sf is  degrees.', name='get_weather', id='46c8fdea-2b44-48b3-9c7f-c3a9c3cb467b', tool_call_i

## 4- Web Search Agent

Now that you know how to use LangChain with tools, let's build something useful. Instead of a toy get_weather tool, let create an agent that searches the web and answers questions using real results. In the next section, you will create a [DuckDuckGo](https://github.com/deedy5/ddgs) search tool and wire it into a ReAct agent.

In [75]:
# ---------------------------------------------------------
# Step 1: Write a web search tool
# ---------------------------------------------------------
# Steps:
#   1. Write a helper function (e.g., search_web) that:
#        • Takes a query string
#        • Uses DuckDuckGo (DDGS) to fetch top results (titles + URLs)
#        • Returns them as a formatted string
#   2. Wrap it with the @tool decorator to make it available to LangChain.


from ddgs import DDGS
from langchain_core.tools import tool

@tool
def search_web(query: str) -> str:
    """Search the web using DuckDuckGO for information.
    Returns a formatted string containing the tittles and URLs of the top results.
    """
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=5))
        # results = [r for r in ddgs.text(query, max_resutls=5)]
    
    if not results:
        return "No results found."

     # Format into a string so the LLM can read the links
    return "\n".join([f"{r['title']}: {r['href']}" for r in results])

print(search_web.invoke("What is the latest version of Python?"))



Python (programming language) - Wikipedia: https://en.wikipedia.org/wiki/Python_(programming_language)
Download Python | Python.org: https://www.python.org/downloads/
Latest Python Version (2025) - What's New in Python 3.14?: https://www.liquidweb.com/blog/latest-python-version/
Python: All Releases, End of Life, Release Date - VersionLog: https://versionlog.com/python/
Status of Python versions: https://devguide.python.org/versions/


In [79]:
# ---------------------------------------------------------
# Step 2: Initialize the web-search agent
# ---------------------------------------------------------
# Steps:
#   1. Create an LLM (e.g., ChatOllama).
#   2. Add your `web_search` tool to the tools list.
#   3. Create the agent using create_agent.
#
# Expected:
#   The agent should be ready to accept user queries
#   and use your web search tool when needed.
# ---------------------------------------------------------

from langchain_ollama import ChatOllama
from langchain.agents import create_agent

llm = ChatOllama(
    model = "llama3.2:3b",
    temperature = 0
)
agent = create_agent(
    model=llm,
    tools=[
        get_weather,
        search_web
    ],
    system_prompt="You are a helpful assistant",
)

pprint(agent.invoke(
    {"messages": [
        {"role": "user",
        "content": "what is the weather in sf"},
        {"role": "user",
        "content": "search for braking news"}
     ]}
))


{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='3ea6ed73-6a1f-410d-a339-41f21516e8dc'),
              HumanMessage(content='search for braking news', additional_kwargs={}, response_metadata={}, id='9dbc779e-646e-47af-8813-a3a0beca9392'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2026-02-07T07:32:42.868122Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1180188083, 'load_duration': 88300042, 'prompt_eval_count': 220, 'prompt_eval_duration': 395924292, 'eval_count': 40, 'eval_duration': 679129498, 'logprobs': None, 'model_name': 'llama3.2:3b', 'model_provider': 'ollama'}, id='lc_run--019c3704-6856-7e31-ab1f-78e5559930e4-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'sf', 'unit': ''}, 'id': '1c6fec9c-4861-49d0-bcdf-c41cb428963d', 'type': 'tool_call'}, {'name': 'search_web', 'args': {'query': 'breaking news'}, 'id': '8435eccf-576

In [ ]:
# ---------------------------------------------------------
# Step 3: Test your Ask-the-Web agent using agent.invoke
# ---------------------------------------------------------
result = agent.invoke(
    {"messages": [
        # {"role": "user",
        # "content": "what is the weather in sf"},
        {
            "role": "user",
            "content": "search for breaking news and give me a summary and ground your response in the last 24 hours"
        }
     ]}
)

print(result["messages"][-1].content)

Based on the latest news from the past 24 hours, here is a summary of breaking news:

**Global News**

* The United Nations Security Council has condemned the recent missile strikes by North Korea towards South Korea and Japan. (Source: NBC News)
* A massive wildfire in Greece has forced the evacuation of thousands of people and destroyed hundreds of homes. (Source: CBS News)

**US News**

* The US Federal Reserve has announced a 0.25% interest rate hike, citing inflation concerns. (Source: USA TODAY)
* A major cyberattack on the Colonial Pipeline company has led to a shutdown of fuel supplies along the East Coast. (Source: CNN)

**Local News**

* A devastating earthquake has struck Turkey and Syria, killing hundreds and leaving thousands homeless. (Source: Associated Press News)

Please note that news is constantly evolving, and this summary only reflects the latest updates from the past 24 hours.

Would you like me to provide more information on any of these topics or search for some

## 5- (Optional) MCP: Model Context Protocol

Up to now, every tool you used started as a Python function you wrote and registered yourself. **MCP (Model Context Protocol)** lets you skip that step. Tools come from an external *server*, and your code just connects to it. Think of it like USB for AI tools: any MCP client can plug into any MCP server and immediately use whatever tools it offers.

Below, we connect to `mcp-server-fetch` (a ready-made server that can retrieve any URL) using the Python MCP SDK. We launch the server, discover its tools, and call one, all without writing a single `@tool` function. To learn more, read: https://github.com/modelcontextprotocol/servers/tree/main/src/fetch

> **LangChain integration:** The `langchain-mcp-adapters` package can convert MCP tools into LangChain-compatible tools automatically, so you can drop them straight into a ReAct agent like the ones in section 4.

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
# Create an MCP client session and connect it to mcp-server-fetch.
# Follow this link: https://github.com/modelcontextprotocol/servers/tree/main/src/fetch

"""
YOUR CODE HERE (~10 lines of code)
"""

In [ ]:
from langchain_mcp_adapters.tools import load_mcp_tools

from langchain.agents import create_agent
from langchain_ollama import ChatOllama

# Load the tool using load_mcp_tools
# create agent with llm and tools same as before
# Fetch the content of a website like http://python.org


"""
YOUR CODE HERE (~10 lines of code)
"""

## 6- (Optional) A Minimal UI

[Chainlit](https://chainlit.io/) is a Python library designed specifically for building LLM and agent UIs. It provides:
- Built-in streaming support
- Message history
- Step visualization (see tool calls as they happen)
- No frontend code required

If you are interested, follow Chainlit's documentation to implement a simple UI for your agent. The process typically involves:

1. You write a Python file named `chainlit_app.py` with the agent creation logic as well as UI handlers (e.g.,`@cl.on_message`)
2. Run the file in your terminal with `chainlit run app.py`
3. A web UI opens automatically at `http://localhost:8000`

In [ ]:
%%writefile chainlit_app.py
# ---------------------------------------------------------
# Chainlit Web Search Agent

import chainlit as cl
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, ToolMessage
from ddgs import DDGS


# ---------------------------------------------------------
# Define the web search tool
# ---------------------------------------------------------

"""
YOUR CODE HERE (~5 lines of code)
"""

# ---------------------------------------------------------
# Create the agent (once at startup)
# ---------------------------------------------------------

"""
YOUR CODE HERE (~3 lines of code)
"""

# ---------------------------------------------------------
# Chainlit message handler
# ---------------------------------------------------------
@cl.on_message
async def handle_message(message: cl.Message):
    """Handle user messages and stream agent responses."""
    
    """
    YOUR CODE HERE
    """


# ---------------------------------------------------------
# Welcome message
# ---------------------------------------------------------
@cl.on_chat_start
async def start(): 
    """
    YOUR CODE HERE (~5 lines of code)
    """

## 🎉 Congratulations!

You have built a **web-enabled agent** from scratch: manual tool calling → JSON schemas → LangChain ReAct → web search → MCP → UI.

Next steps:
* Try adding more tools, such as news or finance APIs.
* Experiment with multiple tools, different models, and measure accuracy vs. hallucination.
* Explore the [MCP server registry](https://github.com/modelcontextprotocol/servers) for ready-made tool servers.

👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.